In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

# List of bag file paths for the three cameras
bag_files = [
    "/home/aaronmcafee/Documents/20250407_134312.bag",
    "/home/aaronmcafee/Documents/20250407_134313.bag",
    "/home/aaronmcafee/Documents/20250407_134314.bag"
]
pipelines = []
video_writers_color = []
video_writers_depth = []

# Video parameters (ensure these match the recorded stream resolution and desired FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30
frame_width = 640
frame_height = 480

# Create pipelines for each bag file and initialize video writers
for i, bag_file in enumerate(bag_files):
    config = rs.config()
    # Use enable_all_streams to automatically use the streams recorded in the bag
    config.enable_device_from_file(bag_file)
    config.enable_all_streams()
    
    pipeline = rs.pipeline()
    try:
        pipeline.start(config)
        pipelines.append(pipeline)
        print(f"Started pipeline for {bag_file}")
        
        # Create VideoWriter objects for color and depth streams
        color_filename = f"camera_{i+1}_color.avi"
        depth_filename = f"camera_{i+1}_depth.avi"
        writer_color = cv2.VideoWriter(color_filename, fourcc, fps, (frame_width, frame_height))
        writer_depth = cv2.VideoWriter(depth_filename, fourcc, fps, (frame_width, frame_height))
        video_writers_color.append(writer_color)
        video_writers_depth.append(writer_depth)
    except Exception as e:
        print(f"Failed to start pipeline for {bag_file}: {e}")

# Create an align object to align depth frames to the color stream
align = rs.align(rs.stream.color)

try:
    while True:
        frames_list = []
        # Retrieve frames for each bag file pipeline
        for idx, pipeline in enumerate(pipelines):
            try:
                frames = pipeline.wait_for_frames(timeout_ms=5000)
                # Align depth to color for this set of frames
                aligned_frames = align.process(frames)
                frames_list.append(aligned_frames)
                print(f"Camera {idx+1}: Frame received.")
            except Exception as e:
                print(f"Camera {idx+1}: Error retrieving frame: {e}")
                frames_list.append(None)
        
        # If any camera did not return a frame, we assume the bag has ended or an error occurred.
        if any(frame is None for frame in frames_list):
            print("One or more cameras did not return a frame. Exiting loop.")
            break

        # Process, display, and save frames from each camera
        for idx, aligned_frames in enumerate(frames_list):
            depth_frame = aligned_frames.get_depth_frame()
            color_frame = aligned_frames.get_color_frame()

            if not depth_frame or not color_frame:
                print(f"Camera {idx+1}: Missing depth or color frame. Skipping this iteration.")
                continue

            # Convert frames to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # For visualization, apply a colormap to the depth image
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

            # Write the frames to their respective video files
            video_writers_color[idx].write(color_image)
            video_writers_depth[idx].write(depth_colormap)

            # Optionally, display the frames in windows
            cv2.imshow(f"Camera {idx+1} - Color", color_image)
            cv2.imshow(f"Camera {idx+1} - Depth", depth_colormap)

        # Exit the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop all pipelines and release video writers
    for pipeline in pipelines:
        pipeline.stop()
    for writer in video_writers_color:
        writer.release()
    for writer in video_writers_depth:
        writer.release()
    cv2.destroyAllWindows()


In [ ]:
!ls